# Postprocessing a DeepONet Trained for a Mass-Spring-Damper System 

In [1]:
### Loading Libraries

import os
import sys
import numpy                              as np
import pandas                             as pd

import matplotlib
from matplotlib                           import pyplot as plt
#WORKSPACE_PATH = os.environ['WORKSPACE_PATH']
WORKSPACE_PATH = os.getcwd()+'/../../../../../../'
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from tensorflow                           import keras

import romnet                             as rmnt

In [2]:
### Defining ODE and its Parameters

from scipy.integrate import solve_ivp

def fOrig(t, y):
    return [y[1], -Params[2]/Params[0]*y[0]-Params[1]/Params[0]*y[1]]

def fOrig_Source(t, y):
    return [y[1], -Params[2]/Params[0]*y[0]-Params[1]/Params[0]*y[1]]

Params = np.array([1.0, 0.5, 3.0], dtype=np.float64)

In [3]:
### Input Data

ROMNet_fld    = WORKSPACE_PATH + '/ROMNet/romnet/'
path_to_run_fld  = WORKSPACE_PATH + '/ROMNet/MSD_100Cases_All/DeepONet/Run_1/'

TrainingCases = [0]#[0,2,4,6,8]
TestCases     = [0,2,4]#[0,2]

NSamples      = 100

Nt            = 100
tout          = np.linspace(0.,15.,Nt)
SOLVER        = 'Radau'

LineTypeVec   = ['-',':','--','.-']*10

In [4]:
print("\n[ROMNet]: Reading Input File from: ", path_to_run_fld)
sys.path.insert(0, path_to_run_fld)

print("\n[ROMNet]: Keep Loading Modules and Functions...")
from ROMNet_Input import inputdata

print("\n[ROMNet]: Initializing Input ...")
InputData               = inputdata(WORKSPACE_PATH)


InputData.InputFilePath = path_to_run_fld+'/ROMNet_Input.py'
InputData.train_int_flg = 0
InputData.path_to_run_fld  = path_to_run_fld


[ROMNet]: Reading Input File from:  /Users/sventuri/WORKSPACE/ROMNet/romnet/scripts/postprocessing/MassSpringDamper/DeepONet/../../../../../..//ROMNet/MSD_100Cases_All/DeepONet/Run_1/

[ROMNet]: Keep Loading Modules and Functions...

[ROMNet]: Initializing Input ...


In [5]:
surrogate_type = InputData.surrogate_type
if (surrogate_type == 'FNN-SourceTerms'):
    surrogate_type = 'FNN'

Net   = getattr(rmnt.nn, surrogate_type)

model = rmnt.model.Model_TF(InputData)

if (InputData.phys_system is not None):
    System = getattr(rmnt.pinn.system, InputData.phys_system)
    system = System(InputData)
    
model.build(InputData, None, Net, system)#, loadfile_no='000027')

NN    = model.net


[ROMNet - model_tf.py    ]:   Initializing the ML Model

[ROMNet - model_tf.py    ]:   Building the ML Model

[ROMNet - deeponet.py               ]:   Constructing Deep Operator Network: 
[ROMNet - system_of_components.py   ]:     Mapping Branch-to-Trunk (i.e., self.branch_to_trunk Object):  [0, 0]
[ROMNet - system_of_components.py   ]:     Constructing System of Components: DeepONet
[ROMNet - component.py              ]:       Constructing Component: Branch_1
[ROMNet - sub_component.py          ]:         Constructed Sub-Component: Main with Layers:       ListWrapper([<romnet.nn.building_blocks.normalization.CustomNormalization object at 0x1697e14f0>, <keras.layers.core.dense.Dense object at 0x1698b9760>, <keras.layers.core.dropout.Dropout object at 0x169a127f0>, <keras.layers.core.dense.Dense object at 0x169a5acd0>, <keras.layers.core.dropout.Dropout object at 0x169a12850>, <keras.layers.core.dense.Dense object at 0x169a12f10>, <keras.layers.core.dropout.Dropout object at 0x169a1246

In [6]:
# LName   = 'Trunk_1_HL' + str(len(InputData.TrunkActFun[0]))

# Trunk = keras.Model(NN.inputs, NN.get_layer(LName).output)
# Trunk.summary()

In [7]:
# LName   = 'Branch_'+InputData.BranchVars[0]+'_HL' + str(len(InputData.TrunkActFun[0]))
# '_HL' + str(len(InputData.BranchActFun[0]))

# Branch1 = keras.Model(NN.Model.inputs, NN.Model.get_layer(LName).output)
# Branch1.summary()

In [8]:
# LName   = 'Branch_'+InputData.BranchVars[1]+'_HL' + str(len(InputData.BranchActFun[1]))

# Branch2 = keras.Model(NN.Model.inputs, NN.Model.get_layer(LName).output)
# Branch2.summary()

In [9]:
# ### Evaluating on Training Data

# FileName  = InputData.path_to_data_fld + '/' + InputData.InputFile
# Data      = pd.read_csv(FileName, header=0)
# Input     = Data[InputData.BranchVars].to_numpy()
# VarNames  = list(Data.columns)
# Nu        = Input.shape[1]
# u         = []
# for iu in range(Nu):
#     u.append(np.unique(Input[:,iu]))
# u         = np.stack(u, axis=0).T


# ## Variable to Be Visualized
# VarOI     = 'x'
# for iVar in range(1,len(VarNames)):
#     if (VarNames[iVar] == VarOI):
#         print('iVar = ', iVar)
#         break

        
# for iTraining in TrainingCases:

#     print('u0 = ', u[iTraining,:])

#     ### Generating Data  
#     output     = solve_ivp( fOrig, tout[[0,-1]], u[iTraining,:], method=SOLVER, t_eval=tout, rtol=1.e-15, atol=1.e-10 )
#     tOrig      = tout
#     yMatOrig   = output.y.T
    
#     ### Creating NN Input
#     tPred     = tOrig[...,np.newaxis]
#     Nt        = len(tPred)
#     uPred     = u[iTraining,:]*np.ones((Nt,2))
#     InputPred = np.concatenate((uPred,tPred), axis=1)
    
#     if (InputData.ProbApproach == 'Deterministic') or (NSamples == 1):
    
#         yMat       = NN.predict(InputPred)    
#         yTrunk     = Trunk.predict(InputPred)
        
#         ## Plotting
#         fig = plt.figure(figsize=(16, 12))
#         plt.plot(tOrig, yMatOrig[:,iVar-1],  'k-', label='Data')
#         plt.plot(tPred, yMat[:,iVar-1],      'b-', label='DeepONet')
#         plt.xlabel('t')
#         plt.ylabel(VarOI)
#         plt.legend()
#         plt.show()
        
#     else:
        
#         ### Sampling NN
#         yMat_sum   = np.zeros((yMatOrig.shape[0], yMatOrig.shape[1]))
#         yMat_sqr   = np.zeros((yMatOrig.shape[0], yMatOrig.shape[1]))
#         yTrunk_sum = np.zeros((yMatOrig.shape[0], 32))
#         yTrunk_sqr = np.zeros((yMatOrig.shape[0], 32))

#         for iSample in range(NSamples):
#             yMat      = NN.predict(InputPred)    
#             yMat_sum += yMat
#             yMat_sqr += yMat**2

#             yTrunk      = Trunk.predict(InputPred)
#             yTrunk_sum += yTrunk
#             yTrunk_sqr += yTrunk**2

#         yTrunk_mean = yTrunk_sum/NSamples
#         yTrunk_std  = np.sqrt( yTrunk_sqr/NSamples - yTrunk_mean**2 )
#         yTrunk_p1   = yTrunk_mean + 1.*yTrunk_std
#         yTrunk_m1   = yTrunk_mean - 1.*yTrunk_std
#         yTrunk_p2   = yTrunk_mean + 2.*yTrunk_std
#         yTrunk_m2   = yTrunk_mean - 2.*yTrunk_std
#         yTrunk_p3   = yTrunk_mean + 3.*yTrunk_std
#         yTrunk_m3   = yTrunk_mean - 3.*yTrunk_std


#         yMat_mean = yMat_sum/NSamples
#         yMat_std  = np.sqrt( yMat_sqr/NSamples - yMat_mean**2 )
#         yMat_p1   = yMat_mean + 1.*yMat_std
#         yMat_m1   = yMat_mean - 1.*yMat_std
#         yMat_p2   = yMat_mean + 2.*yMat_std
#         yMat_m2   = yMat_mean - 2.*yMat_std
#         yMat_p3   = yMat_mean + 3.*yMat_std
#         yMat_m3   = yMat_mean - 3.*yMat_std


#         ## Plotting
#         fig = plt.figure(figsize=(16, 12))
#         plt.plot(tOrig, yMatOrig[:,iVar-1],  'k-', label='Data')
#         plt.plot(tPred, yMat_mean[:,iVar-1], 'b-', label=r'DeepONet, $\mu$')
#         plt.plot(tPred, yMat_p3[:,iVar-1],   'b:', label=r'DeepONet, $\mu+3\sigma$')
#         plt.plot(tPred, yMat_m3[:,iVar-1],   'b:', label=r'DeepONet, $\mu-3\sigma$')
#         plt.xlabel('t')
#         plt.ylabel(VarOI)
#         plt.legend()
#         plt.show()

In [10]:
# ### Evaluating Trunk's Last Layer on Training Data

# ## Neuron of Interest for Trunk's Last Layer
# iN  = 4

# if (InputData.ProbApproach == 'Deterministic') or (NSamples == 1):
    
#     fig = plt.figure(figsize=(16, 12))
#     plt.plot(tPred, yTrunk[:,iN], 'b-', label=r'TrunkNet')
#     plt.xlabel('t')
#     plt.ylabel(VarOI)
#     plt.legend()
#     plt.show()
    
# else:
    
#     fig = plt.figure(figsize=(16, 12))
#     plt.plot(tPred, yTrunk_mean[:,iN], 'b-', label=r'TrunkNet, $\mu$')
#     plt.plot(tPred, yTrunk_p3[:,iN],   'b:', label=r'TrunkNet, $\mu+3\sigma$')
#     plt.plot(tPred, yTrunk_m3[:,iN],   'b:', label=r'TrunkNet, $\mu-3\sigma$')
#     plt.xlabel('t')
#     plt.ylabel(VarOI)
#     plt.legend()
#     plt.show()

In [11]:
input_vars  = model.net.input_vars
trunk_vars  = InputData.input_vars['DeepONet']['Trunk']
branch_vars = InputData.input_vars['DeepONet']['Branch']

FileName  = WORKSPACE_PATH + '/ROMNet/Data/MSD_100Cases/Orig/test/ext/Input.csv'# + InputData.InputFile
Data      = pd.read_csv(FileName, header=0)
Input     = Data[branch_vars].to_numpy()
VarNames  = list(Data.columns)
Nu        = Input.shape[1]
u         = []
for iu in range(Nu):
    u.append(np.unique(Input[:,iu]))
u         = np.stack(u, axis=0).T
u

array([[-3.6, -3.6],
       [-2.8, -2.8],
       [-2. , -2. ],
       [-1.2, -1.2],
       [-0.4, -0.4],
       [ 0.4,  0.4],
       [ 1.2,  1.2],
       [ 2. ,  2. ],
       [ 2.8,  2.8],
       [ 3.6,  3.6]])

In [29]:
### Evaluating on Test Data

FileName  = WORKSPACE_PATH + '/ROMNet/Data/MSD_100Cases/Orig/test/ext/Input.csv'# + InputData.InputFile
Data      = pd.read_csv(FileName, header=0)
Input     = Data[branch_vars].to_numpy()
VarNames  = list(Data.columns)
Nu        = Input.shape[1]
u         = []
for iu in range(Nu):
    u.append(np.unique(Input[:,iu]))
u         = np.stack(u, axis=0).T


## Variable to Be Visualized
VarOI     = 'v'
for iVar in range(1,len(VarNames)):
    if (VarNames[iVar] == VarOI):
        print('iVar = ', iVar)
        break

iSim = 0
for iTest in [0,1,2,3,4]:
    
    print('u0 = ', u[iTest,:])

    ### Generating Data  
    output     = solve_ivp( fOrig, tout[[0,-1]], u[iTest,:], method=SOLVER, t_eval=tout, rtol=1.e-15, atol=1.e-10 )
    tOrig      = tout
    yMatOrig   = output.y.T
    
    ### Creating NN Input
    tPred     = tOrig[...,np.newaxis]
    Nt        = len(tPred)
    uPred     = u[iTest,:]*np.ones((Nt,2))
    InputPred = np.concatenate((uPred,tPred), axis=1)
    
    if (InputData.surrogate_type == 'DeepONet') or (NSamples == 1):
        
        yMat      = model.predict(InputPred) 
        
        tLoop         = np.linspace(0.,15.,3)
        yMatLoop      = np.zeros((len(tLoop),2))
        InputPredLoop = np.concatenate([uPred[0,:][np.newaxis,...], (tLoop[0])[np.newaxis,np.newaxis]], axis=1)
        yMatLoop[0,:] = model.predict(InputPredLoop)
        for it, tLoop_ in enumerate(tLoop[1:]):
            InputPredLoop    = np.concatenate([yMatLoop[it,:][np.newaxis,...], (tLoop[it+1]-tLoop[it])[np.newaxis,np.newaxis]], axis=1)
            yMatLoop[it+1,:] = model.predict(InputPredLoop)
        
        ## Plotting
        
        if (iSim == 0):
            fig    = plt.figure(figsize=(16, 12))
            Label1 = 'Data'
            Label2 = 'DeepONet'
            Label3 = 'DeepONet, Iterative'
        else:
            Label1 = None
            Label2 = None
            Label3 = None
    
        plt.plot(tOrig, yMatOrig[:,iVar-1],  'k', label=Label1)
        plt.plot(tPred, yMat[:,iVar-1],      'r', label=Label2)
        plt.plot(tLoop, yMatLoop[:,iVar-1],  'ob', label=Label3)
        plt.xlabel('t')
        plt.ylabel(VarOI)
        plt.legend()
        #plt.show()
        
    else:

        ### Sampling NN
        yMat_sum = np.zeros((yMatOrig.shape[0], yMatOrig.shape[1]))
        yMat_sqr = np.zeros((yMatOrig.shape[0], yMatOrig.shape[1]))

        for iSample in range(NSamples):
            yMat_ = NN.predict(InputPred)    
            yMat_sum += yMat_
            yMat_sqr += yMat_**2

        yMat_mean = yMat_sum/NSamples
        yMat_std  = np.sqrt( yMat_sqr/NSamples - yMat_mean**2 )
        yMat_p1   = yMat_mean + 1.*yMat_std
        yMat_m1   = yMat_mean - 1.*yMat_std
        yMat_p2   = yMat_mean + 2.*yMat_std
        yMat_m2   = yMat_mean - 2.*yMat_std
        yMat_p3   = yMat_mean + 3.*yMat_std
        yMat_m3   = yMat_mean - 3.*yMat_std


        ## Plotting
        fig = plt.figure(figsize=(16, 12))
        plt.plot(tOrig, yMatOrig[:,iVar-1],  'k-', label='Data')
        plt.plot(tPred, yMat_mean[:,iVar-1], 'b-', label=r'DeepONet, $\mu$')
        plt.plot(tPred, yMat_p3[:,iVar-1],   'b:', label=r'DeepONet, $\mu+3\sigma$')
        plt.plot(tPred, yMat_m3[:,iVar-1],   'b:', label=r'DeepONet, $\mu-3\sigma$')
        plt.xlabel('t')
        plt.ylabel(VarOI)
        plt.legend()
        plt.show()
        
    iSim += 1

NameError: name 'a' is not defined

In [13]:
### Evaluating on Test Data

FileName  = WORKSPACE_PATH + '/ROMNet/Data/MSD_100Cases/Orig/test/ext/Input.csv'# + InputData.InputFile
Data      = pd.read_csv(FileName, header=0)
Input     = Data[branch_vars].to_numpy()
VarNames  = list(Data.columns)
Nu        = Input.shape[1]
u         = []
for iu in range(Nu):
    u.append(np.unique(Input[:,iu]))
u         = np.stack(u, axis=0).T


import time

In [14]:
CostVec_Int = []

iSim = 0
for iTest in [0,1,2,3,4]:
    
    print('u0 = ', u[iTest,:])
    
    t0 = time.time()
    ### Generating Data  
    output     = solve_ivp( fOrig, tout[[0,-1]], u[iTest,:], method=SOLVER, t_eval=tout, rtol=1.e-15, atol=1.e-10 )
    t1 = time.time()
    CostVec_Int.append(t1-t0)
    

    iSim += 1

u0 =  [-3.6 -3.6]


/usr/local/lib/python3.8/site-packages/scipy/integrate/_ivp/common.py:47: UserWarning: `rtol` is too low, setting to 2.220446049250313e-14
  warn("`rtol` is too low, setting to {}".format(100 * EPS))


u0 =  [-2.8 -2.8]
u0 =  [-2. -2.]
u0 =  [-1.2 -1.2]
u0 =  [-0.4 -0.4]


In [34]:
CostVec_Loop = []

iTestVec     = [0,1,2,3,4]

tLoop        = np.linspace(0.,15.,3)

iSim = 0
for iTest in iTestVec:
    
    print('u0 = ', u[iTest,:])
    
    t0 = time.time()
    yMatLoop      = np.zeros((len(tLoop),2))
    InputPredLoop = np.concatenate([u[iTest,:][np.newaxis,...], (tLoop[0])[np.newaxis,np.newaxis]], axis=1)
    yMatLoop[0,:] = model.net.predict(InputPredLoop)
    for it, tLoop_ in enumerate(tLoop[1:]):
        InputPredLoop    = np.concatenate([yMatLoop[it,:][np.newaxis,...], (tLoop[it+1]-tout[it])[np.newaxis,np.newaxis]], axis=1)
        yMatLoop[it+1,:] = model.net.predict(InputPredLoop)

    t1 = time.time()
    CostVec_Loop.append(t1-t0)
    
    plt.plot(tLoop, yMatLoop[:,iVar-1], 'b')
    plt.xlabel('t')
    plt.ylabel(VarOI)
    plt.legend()
    #plt.show()
    
    iSim += 1

NameError: name 'a' is not defined

In [16]:
Cost_Loop     = 0.

iTestVec      = [0,1,2,3,4]*10
n_test         = len(iTestVec)
    


InputPredLoop = np.concatenate([u[iTestVec,:], np.array([tLoop[0]]*n_test)[...,np.newaxis]], axis=1)
yMatLoop      = model.net.predict(InputPredLoop)
yMatLoopAll   = []
for it, tLoop_ in enumerate(tLoop[1:]):
    InputPredLoop = np.concatenate([yMatLoop, np.array([tLoop[it+1]-tLoop[it]]*n_test)[...,np.newaxis]], axis=1)
    t0            = time.time()
    yMatLoop      = model.net.predict(InputPredLoop)
    t1            = time.time()
    Cost_Loop    += t1-t0
    yMatLoopAll.append(yMatLoop)



In [30]:
Data = pd.read_csv(path_to_run_fld+'/Training/History.csv')

fig  = plt.figure(figsize=(12,8))
plt.plot(Data['tot_loss'],     label='Training')
plt.plot(Data['val_tot_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Tot. Loss [MSE]')
plt.legend()
plt.yscale('log')


fig  = plt.figure(figsize=(12,8))
plt.plot(Data['pts_loss'],     label='Training')
plt.plot(Data['val_pts_loss'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Data Loss [MSE]')
plt.legend()
plt.yscale('log')
#plt.ylim([1.e-2, 1.e0])
# fig  = plt.figure(figsize=(12,8))
# plt.plot(Data['scs_loss'],     label='Training')
# plt.plot(Data['val_scs_loss'], label='Validation')
# plt.xlabel('Epoch')
# plt.ylabel('S.C.s Loss [MSE]')
# plt.legend()
# plt.yscale('log')

# fig  = plt.figure(figsize=(12,8))
# plt.plot(Data['ics_loss'],     label='Training')
# plt.plot(Data['val_ics_loss'], label='Validation')
# plt.xlabel('Epoch')
# plt.ylabel('I.C.s Loss [MSE]')
# plt.legend()
# plt.yscale('log')

NameError: name 'a' is not defined

In [18]:
FileName   = InputData.path_to_data_fld + '/test/ext/Input.csv'# + InputData.InputFile
DataIn     = pd.read_csv(FileName, header=0)
Input_     = DataIn.to_numpy()
VarNames   = list(DataIn.columns)

FileName   = InputData.path_to_data_fld + '/test/ext/Output.csv'# + InputData.InputFile
Data       = pd.read_csv(FileName, header=0)
Output_    = Data.to_numpy()

In [19]:
import tensorflow as tf

In [20]:
# iVar           = 0

# inputs         = tf.keras.Input(shape=(1,))
# Output_Trunk_x = inputs
# for i in range(len(model.net.TrunkLayersVecs[iVar])):
#     Layer_Trunk_x  = model.net.TrunkLayersVecs[iVar][i]
#     Output_Trunk_x = Layer_Trunk_x(Output_Trunk_x)
# Trunk_x = keras.Model(inputs=inputs, outputs=Output_Trunk_x)
# #Trunk.summary()

# inputs   = tf.keras.Input(shape=(2,))
# Output_x = inputs
# for i in range(len(model.net.BranchLayersVecs[iVar])):
#     Layer_x  = model.net.BranchLayersVecs[iVar][i]
#     Output_x = Layer_x(Output_x)
# Branch_x = keras.Model(inputs=inputs, outputs=Output_x)
# #Branch_x.summary()



# iVar           = 1

# inputs         = tf.keras.Input(shape=(1,))
# Output_Trunk_v = inputs
# for i in range(len(model.net.TrunkLayersVecs[iVar])):
#     Layer_Trunk_v  = model.net.TrunkLayersVecs[iVar][i]
#     Output_Trunk_v = Layer_Trunk_v(Output_Trunk_v)
# Trunk_v = keras.Model(inputs=inputs, outputs=Output_Trunk_v)
# #Trunk.summary()

# inputs   = tf.keras.Input(shape=(2,))
# Output_v = inputs
# for i in range(len(model.net.BranchLayersVecs[iVar])):
#     Layer_v  = model.net.BranchLayersVecs[iVar][i]
#     Output_v = Layer_v(Output_v)
# Branch_v = keras.Model(inputs=inputs, outputs=Output_v)
# #Branch_v.summary()

In [21]:
# tVec            = Input_[:,0]

# output_Trunk_x  = Trunk_x.predict(tVec[...,np.newaxis])
# output_Branch_x = Branch_x.predict(Input_[:,1::])
# output_x        = np.sum(output_Trunk_x * output_Branch_x[:,0:-1], axis=1) + output_Branch_x[:,-1]

# output_Trunk_v  = Trunk_v.predict(tVec[...,np.newaxis])
# output_Branch_v = Branch_v.predict(Input_[:,1::])
# output_v        = np.sum(output_Trunk_v * output_Branch_v[:,0:-1], axis=1) + output_Branch_v[:,-1]

# fig = plt.figure(figsize=(10,6))
# plt.plot(tVec, output_Trunk_x[:,0])
# fig = plt.figure(figsize=(10,6))
# plt.plot(Input_[:,1], output_Branch_x, 'o')

# fig = plt.figure(figsize=(10,6))
# plt.plot(tVec, output_Trunk_v[:,0])
# fig = plt.figure(figsize=(10,6))
# plt.plot(Input_[:,1], output_Branch_v, 'o')

In [22]:
# fig = plt.figure(figsize=(12,8))
# plt.plot(tVec, output_x, 'r-')
# plt.plot(tVec, output_v, 'k-')
# plt.ylim([-6.,6.])

In [23]:
# Input   = {}
# Weights = {}
# for data_id in InputData.n_train.keys():
#     FileName         = InputData.path_to_data_fld + '/train/'+data_id+'/Input.csv'
#     Data             = pd.read_csv(FileName)
#     Input[data_id]   = Data
    
#     FileName         = InputData.path_to_run_fld + '/Training/LossWeights/'+data_id+'_9100.csv'
#     Data             = pd.read_csv(FileName, header=None)
#     Weights[data_id] = Data

In [24]:
# #%matplotlib qt

# data_id = 'ics'

# # fig = plt.figure()
# # ax  = fig.add_subplot(projection='3d')
# # ax.scatter(Input[data_id]['x'], Input[data_id]['v'], Weights[data_id][1])
# # ax.set_xlabel('x')
# # ax.set_ylabel('y')

# fig = plt.figure()
# plt.scatter(Input[data_id]['x'], Weights[data_id][0])
# plt.xlabel('x')

# fig = plt.figure()
# plt.scatter(Input[data_id]['v'], Weights[data_id][1])
# plt.xlabel('v')

In [25]:
# %matplotlib qt

# data_id = 'res'

# fig = plt.figure()
# ax  = fig.add_subplot(projection='3d')
# ax.scatter(Input[data_id]['t'], Input[data_id]['x'], Weights[data_id][1])
# ax.set_xlabel('t')
# ax.set_ylabel('x')

# # fig = plt.figure()
# # plt.scatter(Input[data_id]['t'], Weights[data_id][0])
# # plt.xlabel('t')

# # fig = plt.figure()
# # plt.scatter(Input[data_id]['t'], Weights[data_id][1])
# # plt.xlabel('t')

In [26]:
### Evaluating on Test Data

FileName  = WORKSPACE_PATH + '/ROMNet/Data/MSD_100Cases/Orig/test/ext/Input.csv'# + InputData.InputFile
Data      = pd.read_csv(FileName, header=0)
Input     = Data[branch_vars].to_numpy()
VarNames  = list(Data.columns)
Nu        = Input.shape[1]
u         = []
for iu in range(Nu):
    u.append(np.unique(Input[:,iu]))
u         = np.stack(u, axis=0).T
n_test     = u.shape[0]


## Variable to Be Visualized
VarOI     = 'v'
for iVar in range(1,len(VarNames)):
    if (VarNames[iVar] == VarOI):
        print('iVar = ', iVar)
        break

iSim = 0
for iTest in range(n_test):#[0,1,2,3,4]:
    
#     ### Generating Data  
#     output     = solve_ivp( fOrig, tout[[0,-1]], u[iTest,:], method=SOLVER, t_eval=tout, rtol=1.e-15, atol=1.e-10 )
#     tOrig      = tout
#     yMatOrig   = output.y.T
    
    ### Creating NN Input
    tPred     = tOrig[...,np.newaxis]
    Nt        = len(tPred)
    uPred     = u[iTest,:]*np.ones((Nt,2))
    InputPred = np.concatenate((uPred,tPred), axis=1)

iVar =  2


In [32]:
iVar     = 1

output_1 = model.net.system_of_components['DeepONet'].components['Branch_1'].call(u, model.net.layers_dict, None, None, training=False)
output_2 = model.net.system_of_components['DeepONet'].components['Branch_2'].call(u, model.net.layers_dict, None, None, training=False)

fig = plt.figure(figsize=(12,12))
ax  = fig.add_subplot(projection='3d')
plt.plot(u[:,0], u[:,1], output_1[:,0], 'b-')
plt.plot(u[:,0], u[:,1], output_1[:,1], 'b:')
plt.plot(u[:,0], u[:,1], output_2[:,0], 'g-')
plt.plot(u[:,0], u[:,1], output_2[:,1], 'g:')
#plt.xlim([1.e-6, 1.e-2])
plt.xlabel(r'x')
plt.ylabel(r'v')
plt.ylabel(r'$\alpha_i$')

NameError: name 'a' is not defined

In [31]:
iVar    = 0

output_ = model.net.system_of_components['DeepONet'].components['Trunk'].call(tPred, model.net.layers_dict, None, None, training=False)

fig = plt.figure(figsize=(12,8))
plt.plot(tPred, output_[:,0], 'b')
plt.plot(tPred, output_[:,1], 'g')
plt.xlabel('t')
plt.ylabel(r'$\Psi_{'+str(iVar+1)+'}}$')

NameError: name 'a' is not defined